# About this notebook

The purpose of this notebook is to upload previously acquired gridMet data to Amazon s3. Before running this notebook, you should have previously run notebook 1 in this directory in order to download gridMet data locally, potentially changing the range of years, in order to acquire data for the desired date range.

In [1]:
import boto3
import pandas as pd
import os
import re
import numpy as np
from pprint import pprint

In [2]:
gridMet_yearly_data = [f for f in os.listdir('.') if re.match(r'gridMet_\d{4}.parquet.gz', f)]
pprint(gridMet_yearly_data)

['gridMet_2000.parquet.gz',
 'gridMet_2001.parquet.gz',
 'gridMet_2002.parquet.gz',
 'gridMet_2003.parquet.gz',
 'gridMet_2004.parquet.gz',
 'gridMet_2005.parquet.gz',
 'gridMet_2006.parquet.gz',
 'gridMet_2007.parquet.gz',
 'gridMet_2008.parquet.gz',
 'gridMet_2009.parquet.gz',
 'gridMet_2010.parquet.gz',
 'gridMet_2011.parquet.gz',
 'gridMet_2012.parquet.gz',
 'gridMet_2013.parquet.gz',
 'gridMet_2014.parquet.gz',
 'gridMet_2015.parquet.gz',
 'gridMet_2016.parquet.gz',
 'gridMet_2017.parquet.gz',
 'gridMet_2018.parquet.gz',
 'gridMet_2019.parquet.gz']


In [3]:
%%time
df = pd.DataFrame()
for file in gridMet_yearly_data:
    df = pd.concat([df, pd.read_parquet(file)])
print(df.shape)

(10168560, 14)
Wall time: 15.2 s


In [4]:
df.head()

precipitation_amount_mm  \
date       latitude  longitude                              
2000-01-01 33.566667 -117.975000                      0.0   
                     -117.933333                      0.0   
                     -117.891667                      0.0   
                     -117.850000                      0.0   
                     -117.808333                      0.0   

                                  relative_humidity_%  \
date       latitude  longitude                          
2000-01-01 33.566667 -117.975000                 40.5   
                     -117.933333                 40.9   
                     -117.891667                 41.2   
                     -117.850000                 40.8   
                     -117.808333                 38.2   

                                  specific_humidity_kg/kg  \
date       latitude  longitude                              
2000-01-01 33.566667 -117.975000                  0.00600   
                     -117.933333                  0.00593   
                     -117.891667                  0.00580   
                     -117.850000                  0.00577   
                     -117.808333                  0.00547   

                                  surface_downwelling_shortwave_flux_in_air_W m-2  \
date       latitude  longitude                                                      
2000-01-01 33.566667 -117.975000                                            139.7   
                     -117.933333                                            136.6   
                     -117.891667                                            133.2   
                     -117.850000                                            129.8   
                     -117.808333                                            126.5   

                                  wind_from_direction_Degrees Clockwise from north  \
date       latitude  longitude                                                       
2000-01-01 33.566667 -117.975000                                             222.0   
                     -117.933333                                             222.0   
                     -117.891667                                             222.0   
                     -117.850000                                             218.0   
                     -117.808333                                             218.0   

                                  wind_speed_m/s  max_air_temperature_K  \
date       latitude  longitude                                            
2000-01-01 33.566667 -117.975000             2.1                  292.0   
                     -117.933333             2.1                  291.9   
                     -117.891667             2.3                  292.1   
                     -117.850000             2.4                  292.0   
                     -117.808333             2.5                  292.6   

                                  min_air_temperature_K  \
date       latitude  longitude                            
2000-01-01 33.566667 -117.975000                  282.2   
                     -117.933333                  282.2   
                     -117.891667                  282.1   
                     -117.850000                  281.4   
                     -117.808333                  281.0   

                                  burning_index_g_Unitless  \
date       latitude  longitude                               
2000-01-01 33.566667 -117.975000                      31.0   
                     -117.933333                      32.0   
                     -117.891667                      33.0   
                     -117.850000                      34.0   
                     -117.808333                      37.0   

                                  dead_fuel_moisture_100hr_Percent  \
date       latitude  longitude                                       
2000-01-01 33.566667 -117.975000                              12

In [5]:
df.describe()

,precipitation_amount_mm,relative_humidity_%,specific_humidity_kg/kg,surface_downwelling_shortwave_flux_in_air_W m-2,wind_from_direction_Degrees Clockwise from north,wind_speed_m/s,max_air_temperature_K,min_air_temperature_K,burning_index_g_Unitless,dead_fuel_moisture_100hr_Percent,dead_fuel_moisture_1000hr_Percent,energy_release_component-g_Unitless,potential_evapotranspiration_mm,mean_vapor_pressure_deficit_kPa
count,6.501450e+06,6.501450e+06,6.501450e+06,6.501450e+06,6.501450e+06,6.501450e+06,6.501450e+06,6.501450e+06,6.501450e+06,6.501450e+06,6.501450e+06,6.501450e+06,6.501450e+06,6.501450e+06
mean,8.502061e-01,3.011313e+01,6.529446e-03,2.386129e+02,2.264572e+02,2.979890e+00,2.976179e+02,2.834824e+02,3.709669e+01,1.203044e+01,1.276973e+01,5.019266e+01,4.326740e+00,1.355720e+00
std,4.372622e+00,1.777520e+01,2.913681e-03,8.001784e+01,7.745626e+01,1.259263e+00,7.789443e+00,6.112466e+00,1.911510e+01,4.546625e+00,4.040355e+00,2.097613e+01,1.935780e+00,9.774859e-01
min,0.000000e+00,1.000000e-01,2.400000e-04,4.500000e+00,0.000000e+00,4.000000e-01,2.686000e+02,2.575000e+02,0.000000e+00,1.900000e+00,3.400000e+00,0.000000e+00,2.000000e-01,0.000000e+00
25%,0.000000e+00,1.540000e+01,4.330000e-03,1.651000e+02,1.960000e+02,2.200000e+00,2.921000e+02,2.791000e+02,2.700000e+01,8.200000e+00,9.700000e+00,3.400000e+01,2.700000e+00,6.300000e-01
50%,0.000000e+00,2.680000e+01,6.220000e-03,2.461000e+02,2.410000e+02,2.700000e+00,2.970000e+02,2.834000e+02,3.700000e+01,1.210000e+01,1.290000e+01,4.700000e+01,4.200000e+00,1.090000e+00
75%,0.000000e+00,4.270000e+01,8.460000e-03,3.129000e+02,2.770000e+02,3.500000e+00,3.031000e+02,2.880000e+02,5.000000e+01,1.540000e+01,1.550000e+01,6.600000e+01,5.700000e+00,1.830000e+00
max,2.871000e+02,1.000000e+02,2.392000e-02,3.777000e+02,3.600000e+02,1.620000e+01,3.227000e+02,3.074000e+02,1.510000e+02,3.060000e+01,3.080000e+01,1.100000e+02,1.320000e+01,7.170000e+00


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 10168560 entries, (Timestamp('2000-01-01 00:00:00'), 33.56666666666667, -117.97499996666667) to (Timestamp('2019-12-31 00:00:00'), 32.400000000000006, -116.01666663333334)
Data columns (total 14 columns):
 #   Column                                            Dtype  
---  ------                                            -----  
 0   precipitation_amount_mm                           float64
 1   relative_humidity_%                               float64
 2   specific_humidity_kg/kg                           float64
 3   surface_downwelling_shortwave_flux_in_air_W m-2   float64
 4   wind_from_direction_Degrees Clockwise from north  float64
 5   wind_speed_m/s                                    float64
 6   max_air_temperature_K                             float64
 7   min_air_temperature_K                             float64
 8   burning_index_g_Unitless                          float64
 9   dead_fuel_moisture_100hr_Percent               

In [7]:
np.sum(df.isna())

precipitation_amount_mm                             3667110
relative_humidity_%                                 3667110
specific_humidity_kg/kg                             3667110
surface_downwelling_shortwave_flux_in_air_W m-2     3667110
wind_from_direction_Degrees Clockwise from north    3667110
wind_speed_m/s                                      3667110
max_air_temperature_K                               3667110
min_air_temperature_K                               3667110
burning_index_g_Unitless                            3667110
dead_fuel_moisture_100hr_Percent                    3667110
dead_fuel_moisture_1000hr_Percent                   3667110
energy_release_component-g_Unitless                 3667110
potential_evapotranspiration_mm                     3667110
mean_vapor_pressure_deficit_kPa                     3667110
dtype: int64

In [8]:
s3_url = 'dse-cohort5-group5'
s3 = boto3.client("s3")
all_objects = s3.list_objects(Bucket=s3_url)
all_objects

{'ResponseMetadata': {'RequestId': '09FE33CDDF4788E8',
  'HostId': 'iUDfMPonCvu9NY5xFBNJ89DxKQ/2gDFtxtAo9g1W68KLLPCaRSN0/O0VSSIveBzYOBRbpgb0UsU=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'iUDfMPonCvu9NY5xFBNJ89DxKQ/2gDFtxtAo9g1W68KLLPCaRSN0/O0VSSIveBzYOBRbpgb0UsU=',
   'x-amz-request-id': '09FE33CDDF4788E8',
   'date': 'Sun, 23 Feb 2020 03:00:48 GMT',
   'x-amz-bucket-region': 'us-west-1',
   'content-type': 'application/xml',
   'transfer-encoding': 'chunked',
   'server': 'AmazonS3'},
  'RetryAttempts': 1},
 'IsTruncated': False,
 'Marker': '',
 'Contents': [{'Key': 'test/',
   'LastModified': datetime.datetime(2020, 2, 14, 2, 26, 50, tzinfo=tzutc()),
   'ETag': '"d41d8cd98f00b204e9800998ecf8427e"',
   'Size': 0,
   'StorageClass': 'STANDARD',
   'Owner': {'DisplayName': 'kcoakley+cohort5group5',
    'ID': 'fe3a1755292b139ad5397ba537aa0aff32ef6ebba07fe45dcbe486d81e1ccc11'}}],
 'Name': 'dse-cohort5-group5',
 'Prefix': '',
 'MaxKeys': 1000,
 'EncodingType': 'url'}

In [9]:
s3_url = "s3://dse-cohort5-group5/wildfire_capstone/gridMet.parquet.gz"
df.to_parquet(s3_url, compression="gzip")